In [92]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from dotenv import dotenv_values
from langgraph.checkpoint.memory import MemorySaver
import json

memory = MemorySaver()

In [93]:
config = dotenv_values(".env")

In [94]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

In [95]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=config["OPEN_AI_KEY"])

In [96]:
PLAN_PROMPT = """You are an expert Microsoft PowerPoint writer tasked with writing a high level outline for a successful business presentation. \
Write such an outline for the user provided topic. Give an outline of the PowerPoint along with any relevant notes \
or instructions for the sections."""

In [97]:
RESEARCH_PLAN_PROMPT = """You are a researcher charged with providing information that can \
be used when creating a business PowerPoint presentation. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [98]:
WRITER_PROMPT = """You are an Business Consultant assistant tasked with developing excellent PowerPoint presentations.\
Generate the best presentation possible for the user's request and the initial outline. \
Create logical sections for the presentation with concise content with an intention to tell a story. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [99]:
REFLECTION_PROMPT = """You are a partner at a Consulting firm grading a presentation that will be used to review with clients. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [100]:
RESEARCH_CRITIQUE_PROMPT = """You are a researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [101]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [102]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=config["TAVILY_API_KEY"])

In [103]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [104]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    # content = state['plan'] or []
    content = []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [105]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [106]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [107]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [108]:
def powerpoint_node(state: AgentState):
    print("IN POWERPOINT NODE")

In [109]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return "powerpoint"
    return "reflect"

In [110]:
builder = StateGraph(AgentState)

In [111]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)
builder.add_node("powerpoint", powerpoint_node)

In [112]:
builder.set_entry_point("planner")

In [113]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {"powerpoint": "powerpoint", "reflect": "reflect"}
)


In [114]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")
builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")
builder.add_edge("powerpoint", END)

In [115]:
graph = builder.compile()

In [90]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': "How can our clients incorporate GenAI into their workflow?",
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': '**Title: Incorporating GenAI into Your Workflow**\n\n**I. Introduction**\n- Brief overview of GenAI technology\n- Importance of integrating GenAI into business workflows\n\n**II. Understanding GenAI**\n- Explanation of what GenAI is\n- Benefits of using GenAI in business processes\n\n**III. Identifying Opportunities for GenAI Integration**\n- Assessing current workflow processes\n- Identifying areas where GenAI can enhance efficiency and productivity\n\n**IV. Implementing GenAI into Your Workflow**\n- Selecting the right GenAI tools for your business\n- Training employees on how to use GenAI effectively\n\n**V. Case Studies**\n- Real-life examples of companies successfully incorporating GenAI\n- Results and benefits achieved through GenAI integration\n\n**VI. Overcoming Challenges**\n- Common obstacles when implementing GenAI\n- Strategies to overcome challenges and ensure successful integration\n\n**VII. Future Trends in GenAI**\n- Emerging technologies and advan

KeyboardInterrupt: 

In [117]:
thread = {"configurable": {"thread_id": "1"}}
events = graph.stream({
    'task': "How can our clients incorporate GenAI into their workflow?",
    "max_revisions": 2,
    "revision_number": 1}, thread)

for event in events:
    print(event)

{'planner': {'plan': '**Title: Incorporating GenAI into Your Workflow**\n\n**I. Introduction**\n- Brief overview of GenAI technology\n- Importance of incorporating GenAI into business workflows\n\n**II. Understanding GenAI**\n- Explanation of what GenAI is and how it works\n- Benefits of using GenAI in business processes\n\n**III. Identifying Opportunities for GenAI Integration**\n- Assessing current workflow processes\n- Identifying areas where GenAI can be integrated for improved efficiency and productivity\n\n**IV. Implementing GenAI into Your Workflow**\n- Steps for successful integration of GenAI\n- Training and onboarding employees for GenAI adoption\n\n**V. Case Studies**\n- Real-life examples of businesses successfully incorporating GenAI into their workflows\n- Results and benefits achieved from using GenAI\n\n**VI. Best Practices for GenAI Integration**\n- Tips for maximizing the benefits of GenAI in business workflows\n- Common pitfalls to avoid when implementing GenAI\n\n**